In [ ]:
from agent import graph

state = graph.invoke({"messages": [{"role": "user", "content": "Who won the euro 2024"}], "max_research_loops": 3, "initial_search_query_count": 3})

In [ ]:
state

In [ ]:
from IPython.display import Markdown

Markdown(state["messages"][-1].content)

In [ ]:
state = graph.invoke({"messages": state["messages"] + [{"role": "user", "content": "How has the most titles? List the top 5"}]})

In [ ]:
Markdown(state["messages"][-1].content)

In [ ]:
from backend.src.agent.graph import graph
from langchain_core.runnables import RunnableConfig
from backend.src.agent.configuration import Configuration
from langchain_core.messages import HumanMessage
from backend.src.agent.traffic_regulations_tool import REGULATIONS_TEXT, TrafficRegulationsRAG
import os # For API key check

print("\n--- Test: Traffic Regulation Article Query ---")

# Input message for a specific article
input_message_article = "我想查詢道路交通管理處罰條例第3條的內容"
# A simpler query also works due to the regex:
# input_message_article = "道路交通管理處罰條例第3條" 
# Even simpler:
# input_message_article = "第3條" # This should also be caught by the regex

# Prepare the initial state for the graph
initial_state = {"messages": [HumanMessage(content=input_message_article)]}

# Configuration
# Ensure API keys are set in the environment for the test to run
if not os.getenv("GEMINI_API_KEY"):
    print("GEMINI_API_KEY is not set. Skipping test or it might fail if LLM is hit.")
    # Depending on strictness, you might raise an error or skip:
    # raise ValueError("GEMINI_API_KEY must be set for this test") 
config_runnable = RunnableConfig(configurable=Configuration(number_of_initial_queries=1).get_config()) # Reduced queries for faster test

final_result_state = None
intermediate_states = []

print(f"Invoking graph with input: {initial_state}")
for event in graph.stream(initial_state, config=config_runnable):
    print(f"Event: {list(event.keys())}") # Print keys of each event to understand structure
    intermediate_states.append(event)
    if "__end__" in event:
        final_result_state = event["__end__"]
        break
    # Fallback if no __end__ key, take the last event from generate_query or query_traffic_regulations
    # This part is tricky without knowing exact event names from the graph stream
    # For now, we rely on __end__ or the last event if __end__ is not found.
    # A more robust way would be to check for specific node outputs if needed.
    # However, the main assertions are on the final AI message and sources.

if not final_result_state and intermediate_states:
    final_result_state = intermediate_states[-1] # Take the last known event as final state if no __end__

# Assert is_traffic_query (indirectly, by checking if RAG was called)
# Direct check of is_traffic_query from generate_query's output in the stream:
generate_query_output = None
for event_data in intermediate_states:
    if 'generate_query' in event_data:
        generate_query_output = event_data['generate_query']
        break

assert generate_query_output is not None, "generate_query node output not found in stream."
assert generate_query_output.get('is_traffic_query') is True, "is_traffic_query was not set to True for a traffic query."
print("Test Passed: is_traffic_query was correctly set to True.")


# Fetch the actual Article 3 text from the source for assertion
temp_rag = TrafficRegulationsRAG(REGULATIONS_TEXT)
# The RAG tool's query method returns the full string including "找到法規條文："
article_3_rag_output = temp_rag.query("第3條") 
article_3_content_for_assertion = ""
if "找到法規條文：" in article_3_rag_output:
    # Extract just the content part, similar to how RAG tool structures it
    # The RAG output is "找到法規條文：\n{title}\n{content}"
    parts = article_3_rag_output.split("\n", 2)
    if len(parts) > 2:
        article_3_content_for_assertion = parts[1] + "\n" + parts[2] # title + content
    else: # Fallback if split doesn't work as expected
        article_3_content_for_assertion = article_3_rag_output

print(f"Input: {input_message_article}")

ai_message_content = ""
# The final_result_state should be the state of the graph at the END.
# The 'messages' in this state should contain the history, with the last one being from the AI.
if final_result_state and final_result_state.get("messages"):
    last_message = final_result_state["messages"][-1]
    if hasattr(last_message, 'content'):
        ai_message_content = last_message.content
    elif isinstance(last_message, tuple): # ('ai', 'content')
        ai_message_content = last_message[1]
    else:
        ai_message_content = str(last_message) # Fallback
else:
    print("Warning: Could not determine AI message content from final_result_state.")
    print(f"Final result state was: {final_result_state}")


print(f"AI Message Content (first 500 chars): {ai_message_content[:500]}...")

assert article_3_content_for_assertion, "Article 3 content for assertion could not be prepared."
# Check if the core content of Article 3 is present.
# The finalize_answer node might slightly rephrase or summarize.
# A robust check is to see if a significant, unique part of Article 3 is there.
# "本條例用詞，定義如下：" is the start of Article 3.
# "臨時停車：指車輛因上、下人、客，裝卸物品" is a unique part of Article 3, item 10.
assert "本條例用詞，定義如下：" in ai_message_content, "Article 3 definition start not found in AI response."
assert "臨時停車：指車輛因上、下人、客，裝卸物品" in ai_message_content, "Key phrase '臨時停車' from Article 3 not found in AI response."
print("Test Passed: Article 3 content is present in the final AI response.")

# Check sources (should be empty for RAG queries routed to finalize_answer without web_research)
sources = []
if final_result_state:
    sources = final_result_state.get("sources_gathered", [])

print(f"Sources gathered: {sources}")
assert not sources, f"Test Failed: Expected no sources for RAG query, but got {sources}"
print("Test Passed: No sources gathered for RAG query, as expected.")

print("--- Test: Traffic Regulation Article Query Completed ---")


In [ ]:
# Test Case 2: Keyword-based Traffic Regulation Query
# Ensure necessary imports are available if this cell is run independently
# from backend.src.agent.graph import graph
# from langchain_core.runnables import RunnableConfig
# from backend.src.agent.configuration import Configuration
# from langchain_core.messages import HumanMessage
# from backend.src.agent.traffic_regulations_tool import REGULATIONS_TEXT, TrafficRegulationsRAG # Might not be needed directly here
# import os

print("\n--- Test: Keyword-based Traffic Regulation Query ---")

input_message_keyword = "請問紅燈右轉的罰鍰是多少？"
initial_state_keyword = {"messages": [HumanMessage(content=input_message_keyword)]}
# Assuming 'config_runnable' is defined from a previous cell or define it here:
# if 'config_runnable' not in locals():
#     if not os.getenv("GEMINI_API_KEY"):
#         print("GEMINI_API_KEY is not set. This test might fail if LLM is hit.")
#     config_runnable = RunnableConfig(configurable=Configuration(number_of_initial_queries=1).get_config())

final_result_keyword_state = None
is_traffic_query_after_gen_keyword = None
intermediate_states_keyword = []

print(f"Invoking graph with input: {initial_state_keyword}")
for event in graph.stream(initial_state_keyword, config=config_runnable): # Use config_runnable
    #print(f"Keyword Test Event: {list(event.keys())}") # For debugging stream
    intermediate_states_keyword.append(event)
    if "generate_query" in event:
        is_traffic_query_after_gen_keyword = event["generate_query"].get("is_traffic_query")
    if "__end__" in event:
        final_result_keyword_state = event["__end__"]
        break
        
if not final_result_keyword_state and intermediate_states_keyword:
    final_result_keyword_state = intermediate_states_keyword[-1]

print(f"Input: {input_message_keyword}")
print(f"Is traffic query after generate_query: {is_traffic_query_after_gen_keyword}")
assert is_traffic_query_after_gen_keyword is True, "Test Failed: Keyword query was not identified as a traffic query."
print("Test Passed: Keyword query correctly identified as a traffic query.")

ai_message_content_keyword = ""
if final_result_keyword_state and final_result_keyword_state.get("messages"):
    last_message = final_result_keyword_state["messages"][-1]
    if hasattr(last_message, 'content'):
        ai_message_content_keyword = last_message.content
    elif isinstance(last_message, tuple):
        ai_message_content_keyword = last_message[1]
    else:
        ai_message_content_keyword = str(last_message)
else:
    print(f"Warning: Could not determine AI message content from final_result_keyword_state: {final_result_keyword_state}")

print(f"AI Message Content (Keyword - first 500 chars): {ai_message_content_keyword[:500]}...")

# Check for content related to Article 53 (fines for running red light / red light right turn)
# The RAG tool should find "第53條" which details this.
# "汽車駕駛人，行經有燈光號誌管制之交岔路口闖紅燈者，處新臺幣一千八百元以上五千四百元以下罰鍰。"
# "前項紅燈右轉行為者，處新臺幣六百元以上一千八百元以下罰鍰。"
assert "第五十三條" in ai_message_content_keyword or \
       ("闖紅燈" in ai_message_content_keyword and "罰鍰" in ai_message_content_keyword) or \
       ("紅燈右轉" in ai_message_content_keyword and "罰鍰" in ai_message_content_keyword and \
        ("六百元" in ai_message_content_keyword or "一千八百元" in ai_message_content_keyword or "5400" in ai_message_content_keyword or "1800" in ai_message_content_keyword or "600" in ai_message_content_keyword) # Adding numeric checks
       ), \
       "Test Failed: Expected content related to red light right turn fines (Article 53) not found in AI response."
print("Test Passed: Keyword-based traffic query response seems correct.")

sources_keyword = []
if final_result_keyword_state:
    sources_keyword = final_result_keyword_state.get("sources_gathered", [])
    
print(f"Sources gathered (Keyword): {sources_keyword}")
assert not sources_keyword, f"Test Failed: Expected no sources for RAG query, but got {sources_keyword}"
print("Test Passed: No sources gathered for keyword RAG query.")

print("--- Test: Keyword-based Traffic Regulation Query Completed ---")


In [ ]:
# Test Case 3: Non-Traffic (General Web Search) Query
print("\n--- Test: Non-Traffic (Web Search) Query ---")

input_message_web = "今天的台北天氣如何？"
initial_state_web = {"messages": [HumanMessage(content=input_message_web)]}
# Assuming 'config_runnable' is defined from a previous cell or define it here

final_result_web_state = None
is_traffic_query_after_gen_web = None
web_research_node_output_found = False 
intermediate_states_web = []

print(f"Invoking graph with input: {initial_state_web}")
for event in graph.stream(initial_state_web, config=config_runnable): # Use config_runnable
    #print(f"Web Test Event: {list(event.keys())}") # For debugging stream
    intermediate_states_web.append(event)
    if "generate_query" in event:
        is_traffic_query_after_gen_web = event["generate_query"].get("is_traffic_query")
    # Check if web_research node produced a result. The result is a dict with 'web_research_result'
    if "web_research" in event and event["web_research"].get("web_research_result"):
         web_research_node_output_found = True
    if "__end__" in event:
        final_result_web_state = event["__end__"]
        break

if not final_result_web_state and intermediate_states_web:
    final_result_web_state = intermediate_states_web[-1]


print(f"Input: {input_message_web}")
print(f"Is traffic query after generate_query: {is_traffic_query_after_gen_web}")
assert is_traffic_query_after_gen_web is False, "Test Failed: Web query was misidentified as a traffic query."
print("Test Passed: Web query correctly identified as non-traffic.")

ai_message_content_web = ""
if final_result_web_state and final_result_web_state.get("messages"):
    last_message = final_result_web_state["messages"][-1]
    if hasattr(last_message, 'content'):
        ai_message_content_web = last_message.content
    elif isinstance(last_message, tuple):
        ai_message_content_web = last_message[1]
    else:
        ai_message_content_web = str(last_message)
else:
    print(f"Warning: Could not determine AI message content from final_result_web_state: {final_result_web_state}")


print(f"AI Message Content (Web - first 500 chars): {ai_message_content_web[:500]}...")
assert "天氣" in ai_message_content_web or \
       "weather" in ai_message_content_web.lower() or \
       "台北" in ai_message_content_web or \
       "Taipei" in ai_message_content_web, \
       "Test Failed: Expected weather-related content not found in AI response for web query."
print("Test Passed: Web search query response seems correct.")

sources_web = []
if final_result_web_state:
    sources_web = final_result_web_state.get("sources_gathered", [])

print(f"Sources gathered (Web): {sources_web}")
# For a web query, we expect the web_research node to have been called.
# And typically, it should find some sources unless the search is very obscure or fails.
assert web_research_node_output_found, "Test Failed: Web research node does not seem to have produced output for a web query."
# We can also check if sources_web is populated, but web_research_node_output_found is a more direct check that the node ran.
# assert sources_web, "Test Failed: Expected sources for web query, but got none. (Might be a flaky assertion if web search truly finds nothing)."
print("Test Passed: Web research path appears to have been taken and produced output.")

print("--- Test: Non-Traffic (Web Search) Query Completed ---")
